## Q1. Running Ollama with Docker

Let's run ollama with Docker. We will need to execute the 
same command as in the lectures:

```bash
docker run -it \
    --rm \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

What's the version of ollama client? 

```bash
docker exec -it ollama bash
ollama -v
```
### Answer:

ollama version is 0.1.48

## Q2. Downloading an LLM 

We will donwload a smaller LLM - gemma:2b. 

Again let's enter the container and pull the model:

```bash
ollama pull gemma:2b
```

In docker, it saved the results into `/root/.ollama`

We're interested in the metadata about this model. You can find
it in `models/manifests/registry.ollama.ai/library`

What's the content of the file related to gemma?

### Answer:

```bash
    cd /root/ollama/models/manifests/registry.ollama.ai/library/gemma
    cat 2b
```
Output:

{"schemaVersion":2,"mediaType":"application/vnd.docker.distribution.manifest.v2+json","config":{"mediaType":"application/vnd.docker.container.image.v1+json","digest":"sha256:...

## Q3. Running the LLM

Test the following prompt: "10 * 10". What's the answer?

### Answer:

```bash
ollama run gemma:2b
10 * 10
```

Output:

```
Sure, the model you requested is:

10 * 10

It will evaluate to 100.
```

## Q4. Donwloading the weights 

We don't want to pull the weights every time we run
a docker container. Let's do it once and have them available
every time we start a container.

First, we will need to change how we run the container.

Instead of mapping the `/root/.ollama` folder to a named volume,
let's map it to a local directory:

```bash
mkdir ollama_files

docker run -it \
    --rm \
    -v ./ollama_files:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

Now pull the model:

```bash
docker exec -it ollama ollama pull gemma:2b 
```

What's the size of the `ollama_files/models` folder? 

* 0.6G
* 1.2G
* 1.7G
* 2.2G

Hint: on linux, you can use `du -h` for that.

### Answer:
```bash
cd ollama_files
du -h models
```

Output:

```
8.0K    models/manifests/registry.ollama.ai/library/gemma
12K     models/manifests/registry.ollama.ai/library
16K     models/manifests/registry.ollama.ai
20K     models/manifests
1.6G    models/blobs
1.6G    models
```

## Q5. Adding the weights 

Let's now stop the container and add the weights 
to a new image

For that, let's create a `Dockerfile`:

```dockerfile
FROM ollama/ollama

COPY ...
```

What do you put after `COPY`?

### Answer:

```dockerfile
FROM ollama/ollama

COPY ollama_files /root/.ollama
```


## Q6. Serving it 

Let's build it:

```bash
docker build -t ollama-gemma2b .
```

And run it:

```bash
docker run -it --rm -p 11434:11434 ollama-gemma2b
```

We can connect to it using the OpenAI client

Let's test it with the following prompt:

```python
prompt = "What's the formula for energy?"
```

Also, to make results reproducible, set the `temperature` parameter to 0:

```bash
response = client.chat.completions.create(
    #...
    temperature=0.0
)
```

How many completion tokens did you get in response?

* 304
* 604
* 904
* 1204

### Answer:

```python
response = client.chat.completions.create(
        model='gemma:2b',
        temperature=0.0,
        messages=[{"role": "user", "content": prompt}]
    )
    
response.usage.completion_tokens
```
Output:

281